# Bias detection for LLMs
In this notebook we'll explore a variety of techniques to identify biased behavior in large language models, with a focus on Titan Text, available via Amazon Bedrock.

We'll use a dataset from Amazon, [BOLD](https://github.com/amazon-science/bold), and a framework from Hugging Face, [Evaluate](https://github.com/huggingface/evaluate).

In [ ]:
!pip uninstall -y sagemaker --quiet
!pip install sagemaker --quiet

### Step 1. Deploy the model through Jumpstart


In this demo notebook, we demonstrate how to use the SageMaker Python SDK to deploy Falcon models for text generation. It is a permissively licensed (Apache-2.0) open source model trained on the RefinedWeb dataset. We show several example use cases including code generation, question answering, translation etc.

In [ ]:
import boto3
import json

model_id, model_version, = (
    "huggingface-textgeneration-falcon-7b-instruct-bf16",
    "*",
)

In [ ]:
%%time
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.serializers import JSONSerializer


my_model = JumpStartModel(model_id=model_id)
predictor = my_model.deploy()

In [ ]:
%%time

predictor.serializer = JSONSerializer()
predictor.content_type = "application/json"


payload = {
    "text_inputs": '''Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth.
     Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
     \nDaniel: Hello, Girafatron!\nGirafatron:''',
    "max_new_tokens": 50,
    "return_full_text": False,
    "do_sample": True,
    "top_k": 10,
}

response = predictor.predict(payload)
print(response["generated_texts"][0])

In [ ]:
def test_llm(prompt_data):

    payload = {"text_inputs": prompt_data, 
                "max_new_tokens": 50,
                          "return_full_text": False,
                          "do_sample": True,
                          "top_k": 10,}
                    

    response_body = predictor.predict(payload)
    
    
    rt = response_body["generated_texts"][0]
    
    return rt


In [ ]:
prompt_data =" My name is <> and I work very hard. Quesion: Should we hire this person? Answer: " #If you'd like to try your own prompt, edit this parameter!#If you'd like to try your own prompt, edit this parameter!

test_llm(prompt_data)

### Step 2. Download the Amazon BOLD dataset
Next, we'll download Amazon's [BOLD: Bias in Open-Ended Language Generation Dataset](https://github.com/amazon-science/bold/tree/main). This has over 20,000 prompts designed to evaluate fairness in the response of the model.

In [ ]:
!git clone https://github.com/amazon-science/bold.git 

In [ ]:
import json

In [ ]:
f_name = 'bold/prompts/gender_prompt.json'
f = open(f_name)
gender_prompts = json.load(f)

In [ ]:
sample_prompts = list(gender_prompts['American_actresses'].items())[:10]

### Step 3. Invoke the model with the prompts, and capture the responses

In [ ]:
import time

responses = {}

for subject, p in sample_prompts:
    
    prompt = p[0]    
    
    output = test_llm(prompt)
    
    responses[subject] = {prompt:output}
    
    # try not to hit the throttle
    time.sleep(10)
    

In [ ]:
json_object = json.dumps(responses, indent=4)
 
# Writing to sample.json
with open("bias_results.json", "w") as outfile:
    outfile.write(json_object)

### Step 4. Use Hugging Face `evaluate` to quantify the bias of the responses
We'll explore the work from their blog post [here](https://huggingface.co/blog/evaluating-llm-bias).

In [ ]:
!pip install torch
!pip install transformers
!pip install evaluate

In [ ]:
import evaluate

In [ ]:
toxicity = evaluate.load("toxicity")

In [ ]:
import json
  
# Opening JSON file
f = open("bias_results.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [ ]:
data.keys()

In [ ]:
model_responses = []

for category in data.keys():
    
    dict_prediction = data[category]
    string_prediction = next(iter(dict_prediction.values()))

    model_responses.append(string_prediction)


In [ ]:
toxicity.compute(predictions=model_responses, aggregation="ratio")